In [1]:
from math import sqrt
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

from qoptcraft.basis import get_photon_basis
from qoptcraft.state import PureState, MixedState, Fock
from qoptcraft.invariant import can_transition, photon_invariant

In [2]:
def real_to_complex(z: NDArray) -> NDArray:
    return z[:len(z)//2] + 1j * z[len(z)//2:]

def complex_to_real(z: NDArray):
    return np.concatenate((np.real(z), np.imag(z)))

In [3]:
modes = 6
photons = 4
photon_basis = get_photon_basis(modes, photons)
fock_list = [fock for fock in photon_basis if fock[-1] != 1 or fock[-2] != 1]

def invariant(x: NDArray) -> float:
    """Generate a constraint to make the invariant equal to zero.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: sum of square differences between in and out invariants
            for all the input and output states.
    """
    x = real_to_complex(x)
    fock_in_list = [(1, 0, 1, 0, 1, 1), (1, 0, 0, 1, 1, 1), (0, 1, 1, 0, 1, 1), (0, 1, 0, 1, 1, 1)]
    # x[0] = x[0] * 2
    x = x / np.sqrt(np.sum(np.abs(x) ** 2))
    AMP = x[0]
    # AMP = np.sqrt(2 / 27)

    rng = np.random.default_rng(5)

    invariant_constraint = 0
    for _ in range(30):
        coefs_in = np.sqrt(rng.uniform(0, 1, 4)) * np.exp(
            1.0j * rng.uniform(0, 2 * np.pi, 4)
        )
        coefs_in = coefs_in / np.sum(np.abs(coefs_in) ** 2)
        state_in = PureState(fock_in_list, coefs_in)
        inv_in = photon_invariant(state_in, method="reduced")

        coefs_out = AMP * np.diag([1, 1, 1, -1]) @ coefs_in
        # state_out = PureState([*fock_in_list, *fock_list], np.append(coefs_out, x[1:]))
        state_out = PureState([*fock_in_list, *fock_list], np.append(coefs_out, x[1:]))
        inv_out = photon_invariant(state_out, method="reduced")
        invariant_constraint += (inv_in - inv_out) ** 2

    print(f"prob = {np.abs(AMP)**2}")
    print(f"Invariant = {invariant_constraint}\n")

    return invariant_constraint

In [4]:
x_init = np.ones(2 + 2 * len(fock_list))
method = "SLSQP"
constr_prob = {"type": "eq", "fun": lambda x: 1 - x @ x}
(res := minimize(invariant, x_init, method=method, constraints=constr_prob))

prob = 0.008547008547008546
Invariant = 572.3116620511216

prob = 0.008547008673280353
Invariant = 572.3116618693134

prob = 0.008547008545919998
Invariant = 572.3116619297748

prob = 0.008547008545919998
Invariant = 572.3116620385301

prob = 0.008547008545919998
Invariant = 572.3116620386803

prob = 0.008547008545919998
Invariant = 572.3116620385878

prob = 0.008547008545919998
Invariant = 572.3116619859312

prob = 0.008547008545919998
Invariant = 572.3116619859312

prob = 0.008547008545919998
Invariant = 572.3116620555113

prob = 0.008547008545919998
Invariant = 572.3116621431379

prob = 0.008547008545919998
Invariant = 572.3116621430453

prob = 0.008547008545919998
Invariant = 572.3116620768433

prob = 0.008547008545919998
Invariant = 572.3116620768432

prob = 0.008547008545919998
Invariant = 572.3116620557186

prob = 0.008547008545919998
Invariant = 572.3116621431788

prob = 0.008547008545919998
Invariant = 572.3116620792052

prob = 0.008547008545919998
Invariant = 572.311662079205

KeyboardInterrupt: 

In [43]:
np.sum(np.abs(res.x[:2])**2)

0.19112513058894173

In [44]:
x_init = np.ones(2 + 2 * len(fock_list))
method = "L-BFGS-B"
(res := minimize(invariant, x_init, method=method))

prob = 0.008547008547008546
Invariant = 105.1609059490758

prob = 0.008547008631748117
Invariant = 105.16090590865832

prob = 0.00854700854627803
Invariant = 105.16090593366746

prob = 0.00854700854627803
Invariant = 105.16090594766462

prob = 0.00854700854627803
Invariant = 105.16090594766558

prob = 0.00854700854627803
Invariant = 105.16090594768954

prob = 0.00854700854627803
Invariant = 105.16090594085284

prob = 0.00854700854627803
Invariant = 105.16090594085281

prob = 0.00854700854627803
Invariant = 105.16090594985045

prob = 0.00854700854627803
Invariant = 105.16090596110908

prob = 0.00854700854627803
Invariant = 105.16090596113311

prob = 0.00854700854627803
Invariant = 105.16090595255139

prob = 0.00854700854627803
Invariant = 105.16090595255147

prob = 0.00854700854627803
Invariant = 105.16090594985072

prob = 0.00854700854627803
Invariant = 105.16090596113352

prob = 0.00854700854627803
Invariant = 105.16090595213319

prob = 0.00854700854627803
Invariant = 105.160905952133

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 1.819409380059506e-07
        x: [ 9.284e+00  5.983e+00 ...  6.952e-01  4.428e+00]
      nit: 34
      jac: [-8.534e-06  1.927e-06 ...  2.485e-06 -2.599e-07]
     nfev: 8695
     njev: 37
 hess_inv: <234x234 LbfgsInvHessProduct with dtype=float64>

In [45]:
np.sum(np.abs(res.x[:2])**2)

121.986438106895

## Solve unitary equation with sympy

First we create a basis of the unitary matrices